# Projeto 1 - Ciência dos Dados

Nome: Pedro Herique Britto Aragão Andrade

Nome: Thiago Shiguero Kawahara

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [348]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 

In [349]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\thiag\Projeto1-Cdados


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [350]:
filename = 'Pringles.xlsx'

In [351]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')

train.Classificação = train.Classificação.astype('category')
train.Classificação.cat.categories = (['Irrelevante', 'Relevante'])

train.head(5)

,Treinamento,Classificação
0,eu 🤝 comer pringles com danette\n\n combinaç...,Relevante
1,"@englantinez pipoca, bolinha de amendoim ou ba...",Relevante
2,chorando e comendo pringles,Irrelevante
3,leticia podendo me viciar em tanta coisa me vi...,Relevante
4,vontade de uma pringles ughhhhh,Irrelevante


In [352]:
test = pd.read_excel(filename, sheet_name = 'Teste')

test.Classificação = test.Classificação.astype('category')
test.Classificação.cat.categories = (['Irrelevante', 'Relevante'])

test.head(5)

,Teste,Classificação
0,o cara tem um pote de pringles entre as perna ...,Irrelevante
1,@ellixquintx a pringles de cebola maior que ca...,Irrelevante
2,se eu fosse líder eu n ia falar com a pessoa q...,Relevante
3,vou fazer meu vício favorito dos últimos tempo...,Relevante
4,ai to muito puta namoral ja era a minha coca c...,Relevante
5,parece que os pringles que eu gosto não são ma...,Irrelevante
6,inexplicável o tanto que a stax pisa na pringles,Irrelevante
7,tem umas foto de ontem q eu to segurando uma l...,Irrelevante
8,"@marcuseduardop1 kkkkkkk só o caderno msm, que...",Irrelevante
9,@_mfitas_ na receita das pringles a batata dev...,Irrelevante


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.


Pringles é um aperitivo no formato batata chips e vendido em um tubo cilindrico.

Relevante: avaliacoes negativas do produto ou que tivessem informacoes interessantes para uma possivel parceria ou novas ideias de marketing.

Nao relevante: todos que nao fossem relevantes.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [353]:
# funcao para limpar o texto de pontuacoes

def cleanup(text):
    punctuation = '[|/!-.:?;¿,)("]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return limpandinho(limpa_aspas(text_subbed))

In [354]:
# funcao para tirar aspas

def limpa_aspas(text):
    lista=["'", '"'] 
    for i in text:
        if i in lista:
            text = text.replace(i,"")
    return text

In [469]:
# funcao para transformar as palavras abreviadas em seus padroes + deletar palavras sem valor

def limpandinho(text):
    dicio = {"não": ["n","nn","nao","nnn"], 
             "sim": ["ss","s","sss","siim"], 
             "porque": ["pq","motivo"],
             "muito": ["mt"],
             "tudo": ["td","todo"]
            }

    valores=[]
    for lista in dicio.values():
        for elemento in lista:
            valores.append(elemento)

    nova_frase=[]
    l_palavras = text.split()   
    for palavra in l_palavras:
        if palavra in valores:
            for l_dicio in dicio.values():
                if palavra in l_dicio:
                    nova_frase.append(list(dicio.keys())[list(dicio.values()).index(l_dicio)])
                    
        elif palavra[:2] == "kk":
            palavra = "kkk"
            nova_frase.append(palavra)
            
        elif palavra[0] == "@":
            pass
        elif palavra[:4] == "http":
            pass
        elif palavra in ["a","o","de","e","era","em","que","q","oq"]:
            pass
        
        else:
            nova_frase.append(palavra)
    return " ".join(nova_frase)

In [470]:
# separando a tabela de treinamento em relevante e irrelevante

train_i = train.loc[train.Classificação=="Irrelevante",:]

train_r = train.loc[train.Classificação=="Relevante",:]

In [471]:
# limpando (cleanup()), juntando todas as palavras da tabela treinamento (.join()) e individualizando cada palavra (.split())


# para tabela total
l_t=[]
for i in train.Treinamento:
    l_t.append(cleanup(i))
l_t = " ".join(l_t).split()


# para tabela somente relevante
l_r=[]
for i in train_r.Treinamento:
    l_r.append(cleanup(i))   
l_r = " ".join(l_r).split()


# para tabela irrelevante 
l_i=[]
for i in train_i.Treinamento:
    l_i.append(cleanup(i))    
l_i = " ".join(l_i).split()

In [472]:
tab_total = pd.Series(l_t)
tab_total_a = tab_total.value_counts()
tab_total_r = tab_total.value_counts(True)

tab_rel = pd.Series(l_r)
tab_rel_a = tab_rel.value_counts()
tab_rel_r = tab_rel.value_counts(True)

tab_irr = pd.Series(l_i)
tab_irr_a = tab_irr.value_counts()
tab_irr_r = tab_irr.value_counts(True)

In [473]:
# calculo da probabilidade de ser relevante e da probabilidade de ser irrelevante

probR = sum(tab_rel_a)/sum(tab_total_a)
probI = sum(tab_irr_a)/sum(tab_total_a)

In [474]:
# limpando e separando cada frase da tabela teste

l_teste=[]
for i in test.Teste:
    c = cleanup(i)
    l_teste.append(c.split())

In [475]:
# cridando um DataFrame para o maquina preencher

data = {'Tweet':  [],
        'Classificação': [],
        }

novo = pd.DataFrame(data, columns = ['Tweet','Classificação'])

In [476]:
palavras_unicas_total=[]
for i in l_t:
    if i not in palavras_unicas_total:
        palavras_unicas_total.append(i)

In [477]:
# Legenda: T == Tweet

contador = 0
for frase in l_teste:
    probTDadoR = 1
    for palavra in frase:
        # Laplace smoothing
        if palavra in tab_rel_a:
            probTDadoR*=(tab_rel_a[palavra] + 1) / (len(tab_rel) + len(palavras_unicas_total))
        else:
            probTDadoR*=(0+1) / (len(tab_rel) + len(palavras_unicas_total))
            
            
    probTDadoI = 1
    for palavra in frase:
        # Laplace smoothing
        if palavra in tab_irr_a:
            probTDadoI*=(tab_irr_a[palavra] + 1) / (len(tab_irr) + len(palavras_unicas_total))
        else:
            probTDadoI*=(0 + 1) / (len(tab_irr) + len(palavras_unicas_total))
            
    probRdadoT = probTDadoR*probR
    
    probIdadoT = probTDadoI*probI

    if probRdadoT>probIdadoT:
        
        novo.loc[contador,"Tweet"]= " ".join(frase)
        novo.loc[novo.Tweet==" ".join(frase),"Classificação"] = "Relevante"
    
    elif probRdadoT<probIdadoT:
        
        novo.loc[contador,"Tweet"]=" ".join(frase)
        novo.loc[novo.Tweet==" ".join(frase),"Classificação"] = "Irrelevante"
    
    contador+=1

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [478]:
vp = 0
fp = 0
vn = 0
fn = 0

novo["Performance"] = "nada"

for i in range(len(novo)):
    if novo.iloc[i, 1] == test.iloc[i,1]:
        if novo.iloc[i, 1] == "Relevante":
            vp+=1
            novo.iloc[i, 2]="VP"
        else:
            vn+=1
            novo.iloc[i, 2]="VN"
    elif novo.iloc[i, 1] != test.iloc[i,1]:
        if novo.iloc[i, 1] == "Relevante":
            fp+=1
            novo.iloc[i, 2]="FP"
        else:
            fn+=1
            novo.iloc[i, 2]="FN"

In [507]:
P = novo.loc[:,["Performance"]].value_counts(normalize=True).round(4)*100
vp = P["VP"]
fp = P["FP"]
vn = P["VN"]
fn = P["FN"]
print(f"Porcentagem de verdadeiros positivos: {vp:.1f}")
print(f"Porcentagem de falsos positivos: {fp:.1f}")
print(f"Porcentagem de verdadeiros negativos: {vn:.1f}")
print(f"Porcentagem de falsos negativos: {fn:.1f}")

Porcentagem de verdadeiros positivos: 15.0
Porcentagem de falsos positivos: 14.5
Porcentagem de verdadeiros negativos: 56.5
Porcentagem de falsos negativos: 14.0


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**